In [10]:
%cd "/content/drive/MyDrive/IT/Cao hoc/Luan van/Python/"

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/IT/Cao hoc/Luan van/Python/'
d:\My Drive\IT\Cao hoc\Luan van\Python


In [3]:
!pip install .

Processing /content/drive/MyDrive/IT/Cao hoc/Luan van/Python
  Preparing metadata (setup.py) ... done
  Created wheel for PRF: filename=PRF-0.1.dev0-py3-none-any.whl size=13034 sha256=dcf51f1966d5a26cb8d33c5a6e9ca0a4b69067723b8deb2f181ee5b29b64319b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vf8883s3/wheels/55/72/3f/d04142653063b3553a2988a5f80f02eebc0ba3d4943780c56b
Successfully built PRF


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, log_loss, brier_score_loss, recall_score, f1_score, precision_score, ConfusionMatrixDisplay, confusion_matrix, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold   # Dùng để tạo py
from PRF import prf

In [2]:
from sklearn.impute import SimpleImputer

def generate_noisy_dataset(
    n_samples=1000,
    n_features=10,
    n_classes=3,
    weights=[0.5, 0.3, 0.2],
    noise_level=0.1,
    label_noise_ratio=0.3,
    noise_type='uniform',
    n_groups=2,
    missing_rate=0.0,
    random_state=42
):
    """
    Tạo dữ liệu có nhiễu cho PRF theo 4 kiểu: uniform, per_feature, grouped, complex.

    Trả về:
        X: giá trị đặc trưng (n_samples x n_features)
        dX: độ lệch chuẩn của đặc trưng (n_samples x n_features)
        y: nhãn
        py: xác suất nhãn (PMF) (n_samples x n_classes)
    """
    rng = np.random.default_rng(seed=random_state)

    # Tạo dữ liệu cơ bản
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=int(0.6 * n_features),
        n_redundant=int(0.2 * n_features),
        n_classes=n_classes,
        weights=weights,
        flip_y=noise_level * label_noise_ratio,
        class_sep=1.5 - noise_level,
        random_state=random_state
    )

    # Nhiễu đặc trưng (feature noise)
    if noise_type == 'uniform':
        # Mọi điểm dữ liệu đều có mức nhiễu giống nhau (ngay cả giữa các đặc trưng)
        dX = np.full_like(X, fill_value=noise_level * np.std(X) + 0.01)

    elif noise_type == 'per_feature':
        # Mỗi đặc trưng (cột) có độ nhiễu khác nhau nhưng đồng đều trên tất cả các mẫu. Ví dụ: sensor A nhiễu hơn sensor B.
        feature_noise = rng.uniform(0.5, 1.5, size=n_features)
        dX = noise_level * np.abs(X) * feature_noise + 0.01

    elif noise_type == 'grouped':
        # Dữ liệu chia thành n nhóm, mỗi nhóm có pattern nhiễu khác nhau cho các đặc trưng. Ví dụ: mỗi thiết bị đo (hoặc survey) có sai số đặc trưng riêng.
        group_size = n_samples // n_groups
        dX = np.zeros_like(X)
        for g in range(n_groups):
            start = g * group_size
            end = n_samples if g == n_groups - 1 else (g + 1) * group_size
            group_noise = rng.uniform(0.5, 1.5, size=n_features)
            dX[start:end] = noise_level * np.abs(X[start:end]) * group_noise + 0.01

    elif noise_type == 'complex':
        # Mức nhiễu thay đổi theo mỗi đặc trưng của từng mẫu, hoàn toàn không có mẫu số chung. Đây là mô hình thực tế nhất nhưng khó học nhất nếu không khai báo uncertainty.
        object_noise = rng.uniform(0.5, 1.5, size=(n_samples, 1))
        feature_noise = rng.uniform(0.5, 1.5, size=(1, n_features))
        dX = noise_level * np.abs(X) * (object_noise @ feature_noise) + 0.01

    else:
        raise ValueError(f"Không hỗ trợ noise_type = '{noise_type}'")
    
    # Thêm nhiễu vào biến X
    X = X + dX * rng.normal(size=X.shape)
    
    # Tạo dữ liệu bị miss (NaN) ngẫu nhiên
    if missing_rate > 0.0:
        total_values = n_samples * n_features
        n_missing = int(total_values * missing_rate)

        # Chọn ngẫu nhiên các chỉ số để đặt NaN
        missing_indices = rng.choice(total_values, size=n_missing, replace=False)
        row_indices = missing_indices // n_features
        col_indices = missing_indices % n_features

        X[row_indices, col_indices] = np.nan
        dX[np.isnan(X)] = 1.0

        imputer = SimpleImputer(strategy="mean")  # hoặc 'median', 'most_frequent'
        X = imputer.fit_transform(X)
    
    # Tạo py
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    py = np.zeros((len(X), n_classes))

    for train_idx, val_idx in kf.split(X):
        rf = RandomForestClassifier(n_estimators=50, max_depth=8, n_jobs=-1, random_state=41)
        rf.fit(X[train_idx], y[train_idx])
        py[val_idx] = rf.predict_proba(X[val_idx])

    return X, dX, y, py

In [52]:
noisy_type = 1  # Feature Noise
#noisy_type = 2  # Label Noise
#noisy_type  = 3  # Feature Noise + Label Noise
n_samples = 100000
noise_level = 0.1
label_noise_ratio=0.1
n_classes   = 2
weights     = [0.6, 0.4]
missing_rate=0.1

if noisy_type==1:
    X, dX, y, py = generate_noisy_dataset(
        n_samples=n_samples,
        n_features=20,
        n_classes=n_classes,
        weights=weights,
        noise_type='uniform',   # đặc trưng nhiễu kiểu khó
        label_noise_ratio=label_noise_ratio,  # giữ nhãn chính xác
        noise_level=noise_level,         # % nhiễu
        missing_rate = missing_rate
    )
elif noisy_type == 2:
    X, dX, y, py = generate_noisy_dataset(
        n_samples=n_samples,
        n_features=20,
        n_classes=n_classes,
        weights=weights,
        noise_type='uniform',   # giữ đặc trưng sạch
        label_noise_ratio=label_noise_ratio,  # flip_y = noise_level * 1.0
        noise_level=noise_level,         # % nhiễu
        missing_rate = missing_rate
    )
else:
    X, dX, y, py = generate_noisy_dataset(
        n_samples=n_samples,
        n_features=20,
        n_classes=n_classes,
        weights=weights,
        noise_type='complex',   # đặc trưng nhiễu kiểu khó
        label_noise_ratio=label_noise_ratio,
        noise_level=noise_level,         # % nhiễu
        missing_rate = missing_rate
    )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [53]:
model_rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=10, random_state=42)
model_rf.fit(X_train, y_train)

y_pred = model_rf.predict(X_test)
y_proba = model_rf.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
lloss = log_loss(y_test, y_proba)

print(f"RF Accuracy: {accuracy:.2f}")
print(f"RF F1 Score: {f1_macro:.2f}")
print(f"RF Log Loss: {lloss:.4f}")
#print(classification_report(y_test, y_pred, digits=3))

RF Accuracy: 0.92
RF F1 Score: 0.91
RF Log Loss: 0.2577


In [54]:
model_svm = SVC(kernel='rbf', C=1.0, probability=True, random_state=42)
model_svm.fit(X_train, y_train)

y_pred = model_svm.predict(X_test)
y_proba = model_svm.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
lloss = log_loss(y_test, y_proba)

print(f"SVM Accuracy: {accuracy:.2f}")
print(f"SVM F1 Score: {f1_macro:.2f}")
print(f"SVM Log Loss: {lloss:.4f}")
#print(classification_report(y_test, y_pred, digits=3))

SVM Accuracy: 0.98
SVM F1 Score: 0.97
SVM Log Loss: 0.1008


In [55]:
model_xgb = XGBClassifier(n_estimators=100, max_depth=8, min_child_weight=10, learning_rate=0.1, eval_metric='mlogloss', random_state=42)
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
y_proba = model_xgb.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
lloss = log_loss(y_test, y_proba)

print(f"XGB Accuracy: {accuracy:.2f}")
print(f"XGB F1 Score: {f1_macro:.2f}")
print(f"XGB Log Loss: {lloss:.4f}")
#print(classification_report(y_test, y_pred, digits=3))

XGB Accuracy: 0.97
XGB F1 Score: 0.96
XGB Log Loss: 0.1089


In [56]:
X_train, X_test, dX_train, dX_test, py_train, py_test = train_test_split(
    X, dX, py, test_size=0.2, random_state=42, stratify=np.argmax(py, axis=1)
)

model_prf = prf(n_estimators=100, bootstrap=True, max_depth=8, keep_proba=0.8, n_jobs=-1)
model_prf.fit(X=X_train, dX=dX_train, py=py_train)

y_test_bin = py_test.argmax(axis=1)
y_proba_prf = np.array(model_prf.predict_proba(X_test, dX=dX_test))
y_pred_prf = y_proba_prf.argmax(axis=1)
f1_macro = f1_score(y_test_bin, y_pred_prf, average='macro')
lloss = log_loss(y_test_bin, y_proba_prf)

print("PRF Accuracy:", model_prf.score(X_test, dX=dX_test, y=y_test_bin))
print(f"PRF F1 Score: {f1_macro:.2f}")
print(f"PRF Log Loss: {lloss:.4f}")
#print(classification_report(y_test_bin, y_pred_prf, digits=3))

PRF Accuracy: 0.95715
PRF F1 Score: 0.95
PRF Log Loss: 0.2783
